# Capstone Phase 2: AI-Powered Architecture & Data Design

**Project:** StaffAlloc – AI-Powered Project Staffing & Hours Allocation Tool

**Objective:** Apply the Phase 2 guidance from the capstone README to transform the PRD into actionable architectural artifacts. We will prompt an LLM to propose a system architecture, generate PlantUML diagrams, capture key architectural decisions, and produce a normalized relational schema that aligns with the StaffAlloc vision.

---

## 📖 Overview

Phase 2 concentrates on acting as the system architect. Following the README playbook, this notebook guides an AI co-pilot to:

1. Synthesize the architecture from the PRD.
2. Generate diagrams-as-code (PlantUML) artifacts.
3. Capture architecture decision records (ADRs).
4. Produce the relational database schema that will feed later phases.

> All prompts use the authoritative context from `Artifacts/Documentation/prd.md`. Before running the notebook, verify your API credentials and project structure.



## ✅ Before You Begin

Make sure the following prerequisites are satisfied:

1. `.env` file exists in the project root with a valid `GOOGLE_API_KEY` (see `SETUP.md`).
2. Dependencies from `requirements.txt` are installed in your active environment.
3. The Phase 1 artifacts—especially `Artifacts/Documentation/prd.md`—are present.
4. You have network access to call the selected LLM provider.


## Step 0: Install Required Packages

Run this cell first to ensure the current kernel has the libraries needed for architecture generation, diagram rendering, and file management.


In [1]:
import sys
import subprocess

print("Installing required packages in the current kernel environment...")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print("=" * 70)

required_packages = [
    "python-dotenv",
    "google-genai",
    "plantuml",
    "graphviz",
    "pydantic",
]

for package in required_packages:
    print(f"\n📦 Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"   ✓ {package} installed successfully")
    except subprocess.CalledProcessError as exc:
        print(f"   ❌ Failed to install {package}: {exc}")

print("\n" + "=" * 70)
print("✅ Package installation complete!")
print("\nProceed to Step 1.")
print("=" * 70)


Installing required packages in the current kernel environment...
Python executable: c:\Users\640109\T1Capstone\.venv\Scripts\python.exe
Python version: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]

📦 Installing python-dotenv...
   ✓ python-dotenv installed successfully

📦 Installing google-genai...
   ✓ google-genai installed successfully

📦 Installing plantuml...
   ✓ plantuml installed successfully

📦 Installing graphviz...
   ✓ graphviz installed successfully

📦 Installing pydantic...
   ✓ pydantic installed successfully

✅ Package installation complete!

Proceed to Step 1.


## Step 1: Environment Setup

This step mirrors the pattern from Phase 1. We will:

1. Locate the project root and add it to the Python path.
2. Load environment variables from `.env`.
3. Import reusable helper utilities.
4. Initialize the LLM client (default: `gemini-2.5-pro`).


In [2]:
import os
import json

print("📁 Locating project root and preparing environment...")
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    from dotenv import load_dotenv
    print("✓ python-dotenv available")
except ImportError:
    raise RuntimeError("python-dotenv is not installed. Run Step 0 first.")

env_path = os.path.join(project_root, ".env")
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"✓ Loaded environment variables from {env_path}")
else:
    print(f"⚠️ WARNING: .env file missing at {env_path}")

api_key_preview = os.getenv("GOOGLE_API_KEY")
if api_key_preview:
    masked = f"{api_key_preview[:8]}...{api_key_preview[-4:]}" if len(api_key_preview) > 12 else "***"
    print(f"✓ GOOGLE_API_KEY detected ({masked})")
else:
    raise RuntimeError("GOOGLE_API_KEY not found. Add it to your .env file.")

try:
    from utils import (
        setup_llm_client,
        get_completion,
        clean_llm_output,
        save_artifact,
        load_artifact,
    )
    print("✓ Imported utilities from utils.py")
except ImportError as err:
    raise RuntimeError(f"Unable to import project utilities: {err}")

print("\nInitializing LLM client...")
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")
print(f"✅ LLM ready (provider={api_provider}, model={model_name})")


📁 Locating project root and preparing environment...
Project root: c:\Users\640109\T1Capstone
✓ python-dotenv available
✓ Loaded environment variables from c:\Users\640109\T1Capstone\.env
✓ GOOGLE_API_KEY detected (AIzaSyBZ..._C7Q)
✓ Imported utilities from utils.py

Initializing LLM client...


2025-11-05 14:29:41,305 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


✅ LLM ready (provider=google, model=gemini-2.5-pro)


## Step 2: Load Phase 1 Artifacts

Fetch the authoritative PRD so we can embed it in subsequent prompts. The helper utilities persist artifacts under `Artifacts/`, matching the structure used in earlier labs.


In [3]:
prd_path = "Artifacts/Documentation/prd.md"
prd_content = load_artifact(prd_path)

if prd_content:
    print(f"✓ Loaded PRD from {prd_path}")
    print("Preview (first 600 characters):\n")
    print(prd_content[:600])
else:
    raise FileNotFoundError(f"PRD not found at {prd_path}. Confirm Phase 1 artifacts exist.")


✓ Loaded PRD from Artifacts/Documentation/prd.md
Preview (first 600 characters):

# Product Requirements Document: StaffAlloc
**Version:** 1.0
**Date:** November 5, 2025
**Author:** Senior Product Manager
**Status:** Draft

## 1. Executive Summary & Vision

### Product Name
StaffAlloc

### Overview
StaffAlloc is an intelligent, AI-powered staffing management application designed to replace cumbersome and error-prone spreadsheets. It provides project managers, directors, and resource managers with a centralized platform to create projects, allocate employee hours, track budgets, and optimize resource utilization across the entire organization.

### Purpose
The primary purpos


## Step 3: Architecture Challenges

Following the README guidance, each challenge pairs a targeted prompt with automated artifact saving. For Phase 2 we align the architecture with the Day 3 FastAPI + SQLite labs so the full stack can run locally without paid cloud services.

### Challenge 1 – System Architecture Narrative

**Goal:** Produce a comprehensive `architecture.md` document for a local-first prototype. Focus on:

- High-level system overview sized for a single developer workstation
- Responsibilities inside the FastAPI monolith, React client, and supporting utilities
- Local integrations (SQLite, file-based storage, optional local vector database, SMTP dev server)
- Data flow, privacy, and configuration management on a single machine
- Evolution notes for scaling beyond the prototype


In [4]:
architecture_prompt = f"""
You are a senior solutions architect preparing a local-first prototype for the StaffAlloc platform. All runtime must execute on a single developer laptop without paid cloud dependencies. Align technology choices with the Day 3 FastAPI + SQLite labs (FastAPI, Pydantic, SQLAlchemy, SQLite, local file storage, optional local vector DB).

## Product Context
{prd_content}

## Document Structure - Generate ALL sections below in a SINGLE, COMPLETE response:

### 1. Executive Summary (2-3 paragraphs)
- Overview of the local-first prototype approach
- Key technology choices (FastAPI, SQLite, SQLAlchemy, local AI)
- Goals and constraints

### 2. Logical Architecture
- **Client Layer**: React SPA, optional mobile client, CLI tools
- **API Layer**: FastAPI routers organized by domain (users, trips, expenses, settlements, reports, ai)
- **Service Layer**: Business logic, validation, orchestration
- **Data Access Layer**: SQLAlchemy repositories, models
- **Background Jobs**: APScheduler for async tasks
- **Integration Adapters**: Local AI, vector store, file storage, SMTP

### 3. Local Deployment Architecture
- Process topology (uvicorn, React dev server, background worker)
- Port assignments (8000 for API, 5173 for React, etc.)
- Developer workflow (setup, run, test)
- Directory structure

### 4. Data & Storage Strategy
- SQLite database with WAL mode
- Alembic migrations
- Local filesystem for receipts/reports
- Optional Chroma/LanceDB for embeddings
- Configuration via .env

### 5. AI & Automation Features
- Local LLM integration (Ollama/LM Studio)
- Expense categorization pipeline
- RAG chat implementation
- Settlement optimization

### 6. Security & Privacy
- JWT authentication
- Local secrets management
- File permissions
- Data encryption approach

### 7. Testing & Quality
- Pytest with in-memory SQLite
- Integration test strategy
- Linting (ruff, mypy)
- CI/CD with GitHub Actions

### 8. Observability
- Logging (structlog)
- Health check endpoints
- Optional metrics/tracing

### 9. Risks & Migration Path
- Table format with risks, mitigations, and next steps
- Evolution to cloud architecture

CRITICAL: Generate the COMPLETE document with ALL sections above. Do not truncate or summarize. Provide full details for each section. The document should be 800-1200 lines of comprehensive Markdown.

Respond in GitHub-flavored Markdown ready to save as `Artifacts/Documentation/architecture.md`.
"""

print("--- Generating architecture narrative ---")
architecture_doc = get_completion(architecture_prompt, client, model_name, api_provider, temperature=0.3)
print(architecture_doc[:800])

save_artifact(architecture_doc, "Artifacts/Documentation/architecture.md")
print("\n✓ Architecture document saved to Artifacts/Documentation/architecture.md")


--- Generating architecture narrative ---
# StaffAlloc Local-First Prototype: Architecture Document

**Author:** Senior Solutions Architect
**Version:** 1.0
**Date:** Current Date

## 1. Executive Summary

This document outlines the technical architecture for a local-first prototype of the StaffAlloc platform. The primary objective of this prototype is to validate the core user experience and AI-driven features on a single developer laptop, eliminating the need for paid cloud dependencies and enabling rapid, cost-effective iteration. By building a fully functional, self-contained application, we can test the most critical user stories—from the interactive allocation grid to the AI-powered RAG chat—and gather essential feedback before committing to a scalable cloud infrastructure.

Our technology stack is aligned with modern, effic

✓ Architecture document saved to Artifacts/Documentation/architecture.md


### Challenge 2 – PlantUML System Diagram

**Goal:** Capture the local system context as PlantUML diagrams-as-code. Request one diagram for the single-machine context and another for the FastAPI component breakdown so the visuals match the Day 3 lab stack (FastAPI, SQLite, files, local AI, MailHog). Save the PlantUML source for rendering.


In [6]:
plantuml_prompt = f"""
You are an enterprise architect creating diagrams-as-code. Using the StaffAlloc PRD and the updated local-first architecture narrative below, produce TWO COMPLETE PlantUML diagrams in a single response:

## Diagram 1: System Context Diagram
Show the complete local development environment:
- Actors: Traveler, Trip Organizer
- Developer Workstation boundary containing:
  - React Web Client (localhost:5173)
  - FastAPI Backend (localhost:8000)
  - SQLite Database (file)
  - Local File Storage (receipts, reports)
  - Local Vector DB (Chroma/LanceDB)
  - Local AI Runtime (Ollama/LM Studio)
  - MailHog SMTP Server (localhost:8025)
- Show all relationships and communication protocols

## Diagram 2: Backend Component Diagram
Break down the FastAPI application into:
- API Routers (users, trips, expenses, settlements, reports, ai)
- Service Layer (business logic)
- Repository Layer (SQLAlchemy)
- Domain Models (Pydantic + SQLAlchemy)
- Background Jobs (APScheduler)
- Adapters (AI, Vector Store, File Storage, SMTP)
- External stores (SQLite, Filesystem, Chroma, Ollama, MailHog)
- Show all dependencies and data flows

## Product Context
{prd_content}

## Architecture Narrative
{architecture_doc}

CRITICAL REQUIREMENTS:
- Generate BOTH complete diagrams with proper @startuml/@enduml blocks
- Use C4-PlantUML notation (Person, System, Container, Component)
- Include clear titles for each diagram
- Show all relationships with descriptive labels
- Add legends explaining colors/shapes if used
- Output only PlantUML source code, no explanatory text

Provide the complete PlantUML source for both diagrams now.
"""

print("--- Generating PlantUML diagrams ---")
plantuml_source = get_completion(plantuml_prompt, client, model_name, api_provider, temperature=0.2)
plantuml_clean = clean_llm_output(plantuml_source, language="plantuml")
print(plantuml_clean[:600])

save_artifact(plantuml_clean, "Artifacts/Architecture/StaffAlloc_system_diagrams.puml")
print("\n✓ PlantUML saved to Artifacts/Architecture/StaffAlloc_system_diagrams.puml")


--- Generating PlantUML diagrams ---
@startuml StaffAlloc System Context
!include <C4/C4_Context.puml>

title System Context Diagram: StaffAlloc Local Development Environment

LAYOUT_WITH_LEGEND()

Person(pm, "Project Manager", "Manages project staffing, budgets, and timelines. (e.g., Priya)")
Person(director, "Director / Optimizer", "Oversees portfolio health and resource utilization. (e.g., David, Maria)")

System_Boundary(workstation, "Developer Workstation") {
    System(web_client, "React Web Client", "Vite SPA running on localhost:5173. Provides the interactive UI for managing allocations.")
    System(backend, "FastAPI Bac

✓ PlantUML saved to Artifacts/Architecture/StaffAlloc_system_diagrams.puml


### Challenge 3 – Architecture Decision Records (ADRs)

**Goal:** Document critical architectural decisions for the local-first stack. Capture at least three ADRs that mirror the Day 3 lab choices: SQLite persistence, local AI/RAG strategy, and developer workstation runtime/tooling.


In [8]:
adr_prompt = f"""
Act as an experienced software architect. Based on the StaffAlloc PRD and the local-first architecture narrative, author EXACTLY THREE comprehensive Architecture Decision Records (ADRs).

## ADR Template - Use this structure for EACH ADR:

### ADR-00X: [Clear Decision Title]

**Status:** Accepted | Proposed | Deprecated
**Date:** 2025-11-05
**Context:** (2-3 paragraphs explaining the problem, constraints, and forces at play)
**Decision:** (1-2 paragraphs stating what was decided and why)
**Consequences:**
- ✅ Positive consequence 1
- ✅ Positive consequence 2
- ✅ Positive consequence 3
- ⚠️ Negative consequence 1
- ⚠️ Negative consequence 2
**Alternatives Considered:** (Brief list of rejected options)
**Follow-up Actions:** (Specific next steps or review criteria)

## Required ADRs - Generate ALL THREE in full detail:

### ADR-001: SQLite + SQLAlchemy for Local Persistence
- Context: Need database for prototype, Day 3 labs use SQLite
- Decision: Use SQLite with WAL mode, SQLAlchemy ORM, Alembic migrations
- Consequences: Fast setup vs. limited concurrency
- Alternatives: PostgreSQL, in-memory only
- Follow-up: Migration path to PostgreSQL

### ADR-002: Local AI/RAG with Open-Source Models
- Context: Need AI categorization and chat without cloud costs
- Decision: Ollama/LM Studio for LLM, Chroma for vector store
- Consequences: Zero cost vs. lower quality/latency
- Alternatives: OpenAI API, Google Gemini, no AI
- Follow-up: Benchmark quality and create cloud migration plan

### ADR-003: FastAPI Monolith with APScheduler
- Context: Need async background jobs for local prototype
- Decision: Single FastAPI process with APScheduler for jobs
- Consequences: Simple setup vs. limited scalability
- Alternatives: Celery+Redis, separate worker process
- Follow-up: Define criteria for moving to distributed queue

## Product Context
{prd_content}

## Architecture Narrative
{architecture_doc}

CRITICAL: Generate ALL THREE complete ADRs following the template above. Each ADR should be 15-25 lines. Total output should be 200-300 lines of comprehensive Markdown.

Return complete ADRs in Markdown format suitable for `Artifacts/Documentation/adrs.md`.
"""

print("--- Generating ADRs ---")
adrs_raw = get_completion(adr_prompt, client, model_name, api_provider, temperature=0.2)
adrs_clean = clean_llm_output(adrs_raw, language="markdown")
print(adrs_clean[:600])

save_artifact(adrs_clean, "Artifacts/Documentation/adrs.md")
print("\n✓ ADRs saved to Artifacts/Documentation/adrs.md")


--- Generating ADRs ---
### ADR-001: SQLite + SQLAlchemy for Local Persistence

**Status:** Accepted
**Date:** 2025-11-05
**Context:**
The StaffAlloc local-first prototype requires a relational database to store core entities such as projects, employees, and allocations. The primary goal is rapid development and validation on a single developer machine, prioritizing simplicity and minimizing setup overhead. The architecture narrative explicitly calls for a self-contained application without external service dependencies.

The key forces at play are the need for a structured, queryable data store versus the constraint

✓ ADRs saved to Artifacts/Documentation/adrs.md


### Challenge 4 – Relational Schema Generation

**Goal:** Translate the PRD and architecture outputs into a normalized relational database schema (SQLite-compatible) that we will consume in later labs. Include tables, relationships, indexes, and supporting constraints required for StaffAlloc.


In [10]:
schema_prompt = f"""
You are a senior data architect. Using the StaffAlloc PRD and architecture narrative, produce a SQLite-compatible SQL schema. Requirements:

- Use `CREATE TABLE` statements with appropriate data types, primary keys, foreign keys, and `ON DELETE` behaviors.
- Include indexes, unique constraints, and check constraints that enforce the business rules.
- Cover core entities: users, trips, participants, expenses, expense_splits, receipts, payments, AI artefacts (e.g., RAG cache), and audit metadata.
- Add comments (`-- ...`) explaining non-obvious design choices.
- Optimize for analytics by including summary/materialized view candidates or helper tables if helpful.
- End the script with any seed data or helper views only if justified by the PRD.

Respond with pure SQL ready to save into `artifacts/schema.sql`.

## Product Context
{prd_content}

## Architecture Narrative
{architecture_doc}
"""

print("--- Generating SQL schema ---")
schema_sql = get_completion(schema_prompt, client, model_name, api_provider, temperature=0.1)
schema_clean = clean_llm_output(schema_sql, language="sql")
print(schema_clean[:800])

save_artifact(schema_clean, "artifacts/schema.sql")
print("\n✓ Schema saved to artifacts/schema.sql")


--- Generating SQL schema ---
-- StaffAlloc SQLite Schema
-- Version: 1.0
-- Author: Senior Data Architect
--
-- This schema is designed for the StaffAlloc application, optimized for a local-first
-- prototype using SQLite. It covers core entities, relationships, and constraints
-- derived from the PRD and architecture narrative.

-- Enable foreign key constraint enforcement.
PRAGMA foreign_keys = ON;

--------------------------------------------------------------------------------
-- CORE ENTITIES
--------------------------------------------------------------------------------

-- Table: users
-- Stores information about all employees and application users.
-- The system_role column is used for Role-Based Access Control (RBAC).
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    email TEXT NOT NULL UNIQUE,
    full_n

✓ Schema saved to artifacts/schema.sql


## Phase 2 Wrap-Up

You now have:

- `Artifacts/Documentation/architecture.md` – narrative architecture specification.
- `Artifacts/Architecture/StaffAlloc_system_diagrams.puml` – PlantUML diagrams for rendering with PlantUML/Graphviz.
- `Artifacts/Documentation/adrs.md` – curated architecture decision records.
- `artifacts/schema.sql` – normalized relational schema for StaffAlloc.

These artifacts unlock Phase 3 (backend implementation). Continue the capstone by feeding the schema into the FastAPI generation workflows outlined in the Day 3 labs.
